# *Part 3: Modelling*


---

In [96]:
import pandas as pd

In [97]:
HDB = pd.read_csv("train_cleaned.csv")
HDB.head()

,Unnamed: 0,id,town,flat_type,storey_range,flat_model,lease_commence_date,resale_price,Tranc_Year,Tranc_Month,...,pri_sch_nearest_distance,pri_sch_name,pri_sch_affiliation,pri_sch_latitude,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,sec_sch_latitude,sec_sch_longitude
0,0,88471,KALLANG/WHAMPOA,4 ROOM,10 TO 12,Model A,2006,680000.0,2016,5,...,1138.633422,Geylang Methodist School,1,1.317659,103.882504,1138.633422,Geylang Methodist School,224,1.317659,103.882504
1,1,122598,BISHAN,5 ROOM,07 TO 09,Improved,1987,665000.0,2012,7,...,415.607357,Kuo Chuan Presbyterian Primary School,1,1.349783,103.854529,447.894399,Kuo Chuan Presbyterian Secondary School,232,1.350110,103.854892
2,2,170897,BUKIT BATOK,EXECUTIVE,13 TO 15,Apartment,1997,838000.0,2013,7,...,498.849039,Keming Primary School,0,1.345245,103.756265,180.074558,Yusof Ishak Secondary School,188,1.342334,103.760013
3,3,86070,BISHAN,4 ROOM,01 TO 05,Model A,1992,550000.0,2012,4,...,389.515528,Catholic High School,1,1.354789,103.844934,389.515528,Catholic High School,253,1.354789,103.844934
4,4,153632,YISHUN,4 ROOM,01 TO 03,Simplified,1987,298000.0,2017,12,...,401.200584,Naval Base Primary School,0,1.416280,103.838798,312.025435,Orchid Park Secondary School,208,1.414888,103.838335


##   **Rebinning the Storey Range**

In [98]:
# examine the storey range values

HDB["storey_range"].unique()


array(['10 TO 12', '07 TO 09', '13 TO 15', '01 TO 05', '01 TO 03',
       '06 TO 10', '28 TO 30', '19 TO 21', '04 TO 06', '16 TO 18',
       '22 TO 24', '16 TO 20', '34 TO 36', '25 TO 27', '11 TO 15',
       '37 TO 39', '31 TO 33', '43 TO 45', '40 TO 42', '21 TO 25',
       '49 TO 51', '46 TO 48', '26 TO 30', '36 TO 40', '31 TO 35'],
      dtype=object)

In [99]:
# split the storey range values into lowest and highest storey per string

HDB[["low_storey", "high_storey"]] = HDB["storey_range"].str.split("TO", expand=True)
HDB["low_storey"] = HDB["low_storey"].astype(int)
HDB["high_storey"] = HDB["high_storey"].astype(int)


In [100]:
# Compute the average storey (midpoint)
HDB["avg_storey"] = (HDB["low_storey"] + HDB["high_storey"]) / 2

In [101]:
# Define bins and labels for rebinning
bins = [0, 6, 12, 18, 30, 100]  # upper bound of last bin = 100 (covers all)
labels = ["Low (1-6)", "Mid (7-12)", "High (13-18)",
          "Very High (19-30)", "Ultra High (31+)"]

In [102]:
# Create a rebinned category
HDB["new_storey_range"] = pd.cut(HDB["avg_storey"], bins=bins, labels=labels, right=True)
HDB.head()

,Unnamed: 0,id,town,flat_type,storey_range,flat_model,lease_commence_date,resale_price,Tranc_Year,Tranc_Month,...,pri_sch_longitude,sec_sch_nearest_dist,sec_sch_name,cutoff_point,sec_sch_latitude,sec_sch_longitude,low_storey,high_storey,avg_storey,new_storey_range
0,0,88471,KALLANG/WHAMPOA,4 ROOM,10 TO 12,Model A,2006,680000.0,2016,5,...,103.882504,1138.633422,Geylang Methodist School,224,1.317659,103.882504,10,12,11.0,Mid (7-12)
1,1,122598,BISHAN,5 ROOM,07 TO 09,Improved,1987,665000.0,2012,7,...,103.854529,447.894399,Kuo Chuan Presbyterian Secondary School,232,1.350110,103.854892,7,9,8.0,Mid (7-12)
2,2,170897,BUKIT BATOK,EXECUTIVE,13 TO 15,Apartment,1997,838000.0,2013,7,...,103.756265,180.074558,Yusof Ishak Secondary School,188,1.342334,103.760013,13,15,14.0,High (13-18)
3,3,86070,BISHAN,4 ROOM,01 TO 05,Model A,1992,550000.0,2012,4,...,103.844934,389.515528,Catholic High School,253,1.354789,103.844934,1,5,3.0,Low (1-6)
4,4,153632,YISHUN,4 ROOM,01 TO 03,Simplified,1987,298000.0,2017,12,...,103.838798,312.025435,Orchid Park Secondary School,208,1.414888,103.838335,1,3,2.0,Low (1-6)


##   **Mapping of Planning Areas to Regions**

In [103]:
# Define mapping of planning areas to regions (based on URA/HDB definitions)

region_map = {
    "CCR": ["Bukit Timah","Downtown Core","Newton","Orchard","River Valley","Tanglin","Rochor","Outram","Novena"],

    "RCR": ["Bishan","Bukit Merah","Geylang","Kallang","Marine Parade","Queenstown","Toa Payoh"],

    "East": ["Bedok","Pasir Ris","Tampines","Changi"],

    "North-East": ["Ang Mo Kio","Hougang","Punggol","Sengkang","Serangoon"],

    "North": ["Sembawang","Woodlands","Yishun"],

    "West": ["Bukit Batok","Bukit Panjang","Choa Chu Kang","Clementi","Jurong East","Jurong West","Western Water Catchment"]
}

# Reverse lookup and map
region_lookup = {pa: r for r, areas in region_map.items() for pa in areas}

HDB["region"] = HDB["planning_area"].map(region_lookup)

##   **One-Hot Encoding**

<span style="color:purple;">To create the model, we will do one-hot encoding for region, new_storey_range, flat_type and planning_area. </span>

#### *For region:*

In [104]:
# Determine region with highest count

print(HDB["region"].value_counts(dropna=False))
unmapped = HDB.loc[HDB["region"].isna(), "planning_area"].unique().tolist()
print("Unmapped planning areas:", unmapped)

region
North-East    36448
West          36283
RCR           25737
North         25099
East          24331
CCR            2736
Name: count, dtype: int64
Unmapped planning areas: []


<span style="color:purple;">Based on the above value counts, we will remove the North-East column that has the largest representation.</span>

In [105]:
# One-hot encode REGION (drop the region with highest count)
baseline_region = "North-East"
region_dum = pd.get_dummies(HDB["region"], prefix="region", dtype=int)
if f"region_{baseline_region}" in region_dum.columns:
    region_dum.drop(columns=[f"region_{baseline_region}"], inplace=True)

#### *For new_storey_range*

In [106]:
# find the most frequent category for new_storey_range
HDB["new_storey_range"].value_counts()

new_storey_range
Low (1-6)            64459
Mid (7-12)           60293
High (13-18)         19322
Very High (19-30)     5671
Ultra High (31+)       889
Name: count, dtype: int64

<span style="color:purple;">Based on the above value counts, we will remove the Low (1-6) column that has the largest representation.</span>

In [107]:
# One-hot encode the rebinned storey range
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
storey_encoded = encoder.fit_transform(HDB[["new_storey_range"]])
storey_encoded_df = pd.DataFrame(
    storey_encoded,
    columns=encoder.get_feature_names_out(["new_storey_range"])
)

col_to_drop = f"new_storey_range_Low (1-6)"
storey_encoded_df = storey_encoded_df.drop(columns=[col_to_drop])

# preview 5 random rows
storey_encoded_df.sample(n=5, random_state=1)

,new_storey_range_High (13-18),new_storey_range_Mid (7-12),new_storey_range_Ultra High (31+),new_storey_range_Very High (19-30)
95177,0.0,0.0,0.0,0.0
109015,0.0,1.0,0.0,0.0
145236,1.0,0.0,0.0,0.0
38007,0.0,0.0,0.0,0.0
69483,1.0,0.0,0.0,0.0


In [108]:
# generate dummies
flat_type_dummies = pd.get_dummies(HDB["flat_type"], prefix="flat_type")
planning_area_dummies = pd.get_dummies(HDB["planning_area"], prefix="planning_area")

#### *For flat_type*

In [109]:
# for flat_type_dummies, inspect the dataframe. Print 5 random rows.
flat_type_dummies.sample(n=5, random_state=1)

,flat_type_1 ROOM,flat_type_2 ROOM,flat_type_3 ROOM,flat_type_4 ROOM,flat_type_5 ROOM,flat_type_EXECUTIVE,flat_type_MULTI-GENERATION
95177,False,False,False,False,False,True,False
109015,False,False,False,True,False,False,False
145236,False,False,False,False,True,False,False
38007,False,False,True,False,False,False,False
69483,False,False,False,True,False,False,False


In [110]:
# for flat_type, inspect which value has the largest representation to remove
HDB["flat_type"].value_counts()

flat_type
4 ROOM              61136
3 ROOM              39060
5 ROOM              36415
EXECUTIVE           11989
2 ROOM               1896
1 ROOM                 82
MULTI-GENERATION       56
Name: count, dtype: int64

<span style="color:purple;">Based on the above value counts, we will remove the dummy column for flat_type_4 ROOM that has the largest representation.</span>

In [111]:
# remove the dummy column for flat_type_4 ROOM with largest representation
flat_type_dummies.drop(flat_type_dummies.columns[3], axis=1, inplace=True)

#inspect the data again
flat_type_dummies.head()

,flat_type_1 ROOM,flat_type_2 ROOM,flat_type_3 ROOM,flat_type_5 ROOM,flat_type_EXECUTIVE,flat_type_MULTI-GENERATION
0,False,False,False,False,False,False
1,False,False,False,True,False,False
2,False,False,False,False,True,False
3,False,False,False,False,False,False
4,False,False,False,False,False,False


#### *For planning_area*

In [112]:
# for planning_area_dummies, inspect the dataframe. Print 5 random rows.
planning_area_dummies.sample(n=5, random_state=1)

,planning_area_Ang Mo Kio,planning_area_Bedok,planning_area_Bishan,planning_area_Bukit Batok,planning_area_Bukit Merah,planning_area_Bukit Panjang,planning_area_Bukit Timah,planning_area_Changi,planning_area_Choa Chu Kang,planning_area_Clementi,...,planning_area_Rochor,planning_area_Sembawang,planning_area_Sengkang,planning_area_Serangoon,planning_area_Tampines,planning_area_Tanglin,planning_area_Toa Payoh,planning_area_Western Water Catchment,planning_area_Woodlands,planning_area_Yishun
95177,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
109015,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
145236,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
38007,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False
69483,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [113]:
# for planning_area, inspect which value has the largest representation to remove
HDB["planning_area"].value_counts()

planning_area
Jurong West                11451
Woodlands                  11334
Sengkang                   11069
Tampines                   10506
Yishun                     10042
Bedok                       9062
Punggol                     7793
Hougang                     7557
Ang Mo Kio                  6908
Choa Chu Kang               6328
Bukit Merah                 5854
Bukit Batok                 5700
Bukit Panjang               5686
Toa Payoh                   4817
Pasir Ris                   4725
Queenstown                  4103
Geylang                     3986
Sembawang                   3723
Clementi                    3633
Jurong East                 3470
Kallang                     3163
Serangoon                   3121
Bishan                      2871
Novena                      1091
Marine Parade                943
Outram                       759
Rochor                       468
Bukit Timah                  362
Changi                        38
Downtown Core                

<span style="color:purple;">Based on the above value counts, we will remove Jurong West with the largest representation.</span>

In [114]:
# remove the dummy column for Jurong West with largest representation
planning_area_dummies.columns.get_loc("planning_area_Jurong West")
planning_area_dummies.drop(planning_area_dummies.columns[14], axis=1, inplace=True)

#inspect the data again
planning_area_dummies.columns

Index(['planning_area_Ang Mo Kio', 'planning_area_Bedok',
       'planning_area_Bishan', 'planning_area_Bukit Batok',
       'planning_area_Bukit Merah', 'planning_area_Bukit Panjang',
       'planning_area_Bukit Timah', 'planning_area_Changi',
       'planning_area_Choa Chu Kang', 'planning_area_Clementi',
       'planning_area_Downtown Core', 'planning_area_Geylang',
       'planning_area_Hougang', 'planning_area_Jurong East',
       'planning_area_Kallang', 'planning_area_Marine Parade',
       'planning_area_Novena', 'planning_area_Outram',
       'planning_area_Pasir Ris', 'planning_area_Punggol',
       'planning_area_Queenstown', 'planning_area_Rochor',
       'planning_area_Sembawang', 'planning_area_Sengkang',
       'planning_area_Serangoon', 'planning_area_Tampines',
       'planning_area_Tanglin', 'planning_area_Toa Payoh',
       'planning_area_Western Water Catchment', 'planning_area_Woodlands',
       'planning_area_Yishun'],
      dtype='object')

#### *Concatenate all the new one-hot encoded dataframes into original dataframe*

In [115]:
# Concatenate the original DataFrame and the dummy DataFrames
HDB_dummies = pd.concat([HDB, region_dum, flat_type_dummies, planning_area_dummies, storey_encoded_df], axis=1)

# Print 5 random rows
filtered = HDB_dummies.sample(n=5, random_state=1)
filtered.to_csv("filtered_HDB_dummies.csv")

In [116]:
# Check the column names to ensure they are combined
HDB_dummies.columns

Index(['Unnamed: 0', 'id', 'town', 'flat_type', 'storey_range', 'flat_model',
       'lease_commence_date', 'resale_price', 'Tranc_Year', 'Tranc_Month',
       'floor_area_sqft', 'hdb_age', 'commercial', 'market_hawker',
       'total_dwelling_units', 'Latitude', 'Longitude', 'planning_area',
       'Mall_Nearest_Distance', 'Mall_Within_500m', 'Mall_Within_1km',
       'Mall_Within_2km', 'Hawker_Nearest_Distance', 'Hawker_Within_500m',
       'Hawker_Within_1km', 'Hawker_Within_2km', 'hawker_market_stalls',
       'mrt_nearest_distance', 'bus_interchange', 'mrt_interchange',
       'mrt_latitude', 'mrt_longitude', 'bus_stop_nearest_distance',
       'bus_stop_latitude', 'bus_stop_longitude', 'pri_sch_nearest_distance',
       'pri_sch_name', 'pri_sch_affiliation', 'pri_sch_latitude',
       'pri_sch_longitude', 'sec_sch_nearest_dist', 'sec_sch_name',
       'cutoff_point', 'sec_sch_latitude', 'sec_sch_longitude', 'low_storey',
       'high_storey', 'avg_storey', 'new_storey_range', 're

##   **Model 1**

<div style="color:purple;">We will create the 1st model using the feature matrix of the following variables:<br>

- hdb_age
- floor_area_sqft
- flat_type
- storey_range
- planning_area

To kick start, due to overlapping storey_range, we will rebin the storey_range first as part of data wrangling before doing the model.</div>

In [117]:
# Include all dummy variables in feature_cols

from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

feature_cols = ["hdb_age", "floor_area_sqft", "flat_type_1 ROOM",
                "flat_type_2 ROOM", "flat_type_3 ROOM", "flat_type_5 ROOM",
                "flat_type_EXECUTIVE", "flat_type_MULTI-GENERATION",
                "planning_area_Ang Mo Kio", "planning_area_Bedok",
                "planning_area_Bishan", "planning_area_Bukit Batok",
                "planning_area_Bukit Merah", "planning_area_Bukit Panjang",
                "planning_area_Bukit Timah", "planning_area_Changi",
                "planning_area_Choa Chu Kang", "planning_area_Clementi",
                "planning_area_Downtown Core", "planning_area_Geylang",
                "planning_area_Hougang", "planning_area_Jurong East",
                "planning_area_Kallang", "planning_area_Marine Parade",
                "planning_area_Novena", "planning_area_Outram",
                "planning_area_Pasir Ris", "planning_area_Punggol",
                "planning_area_Queenstown", "planning_area_Rochor",
                "planning_area_Sembawang", "planning_area_Sengkang",
                "planning_area_Serangoon", "planning_area_Tampines",
                "planning_area_Tanglin", "planning_area_Toa Payoh",
                "planning_area_Western Water Catchment", "planning_area_Woodlands",
                "planning_area_Yishun", "new_storey_range_High (13-18)",
                "new_storey_range_Mid (7-12)", "new_storey_range_Ultra High (31+)",
                "new_storey_range_Very High (19-30)"]

X = HDB_dummies[feature_cols]
y = HDB_dummies.resale_price

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=123) #random_state is super impt, need to fix the test & train data

model_1 = LinearRegression()
model_1.fit(X_train, y_train) # create a best fit line using train data (x, y)

y_pred = model_1.predict(X_test) # predict the Y values with x test data.

list(zip(feature_cols, model_1.coef_))

[('hdb_age', np.float64(-4213.337124287925)),
 ('floor_area_sqft', np.float64(308.15165092184475)),
 ('flat_type_1 ROOM', np.float64(-94703.23889673832)),
 ('flat_type_2 ROOM', np.float64(-61566.43784192168)),
 ('flat_type_3 ROOM', np.float64(-20175.616536840724)),
 ('flat_type_5 ROOM', np.float64(19371.664539861496)),
 ('flat_type_EXECUTIVE', np.float64(66290.95234067562)),
 ('flat_type_MULTI-GENERATION', np.float64(143701.3721337722)),
 ('planning_area_Ang Mo Kio', np.float64(127402.34695282816)),
 ('planning_area_Bedok', np.float64(109939.13450885279)),
 ('planning_area_Bishan', np.float64(206749.7343397693)),
 ('planning_area_Bukit Batok', np.float64(46532.32570442791)),
 ('planning_area_Bukit Merah', np.float64(219498.1916025161)),
 ('planning_area_Bukit Panjang', np.float64(-9713.753166348863)),
 ('planning_area_Bukit Timah', np.float64(295434.0944611147)),
 ('planning_area_Changi', np.float64(100346.84133950563)),
 ('planning_area_Choa Chu Kang', np.float64(-42998.89317991165)),

In [118]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import numpy as np

model_1_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred)) # compare the RMSE with y_pred and y_test data
print(model_1_rmse)

model_1_mae = metrics.mean_absolute_error(y_test, y_pred)
print(model_1_mae)

model_1_mse = metrics.mean_squared_error(y_test, y_pred)
print(model_1_mse)

61772.467367941674
47956.767266906834
3815837724.723419


##   **Model 2**

<div style="color:purple;">We will create the 2nd model by substituting planning_area with region:<br>

- hdb_age
- floor_area_sqft
- flat_type
- storey_range
- region
</div>

In [119]:
# Include all dummy variables in feature_cols
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
model2_feature_cols = ["hdb_age", "floor_area_sqft", "flat_type_1 ROOM",
                "flat_type_2 ROOM", "flat_type_3 ROOM", "flat_type_5 ROOM",
                "flat_type_EXECUTIVE", "flat_type_MULTI-GENERATION",
                'region_CCR', 'region_East', 'region_North', 'region_RCR', 'region_West', 
                "new_storey_range_High (13-18)",
                "new_storey_range_Mid (7-12)", "new_storey_range_Ultra High (31+)",
                "new_storey_range_Very High (19-30)"]

X = HDB_dummies[model2_feature_cols]
y = HDB_dummies.resale_price

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X, y,random_state=123) 

model_2 = LinearRegression()
model_2.fit(X_train_2, y_train_2) # create a best fit line using train data (x, y)
y_pred_2 = model_2.predict(X_test_2) # predict the Y values with x test data.

list(zip(feature_cols, model_2.coef_))

[('hdb_age', np.float64(-2602.8067402535917)),
 ('floor_area_sqft', np.float64(299.7460764755317)),
 ('flat_type_1 ROOM', np.float64(-87520.11033106191)),
 ('flat_type_2 ROOM', np.float64(-63380.99442069867)),
 ('flat_type_3 ROOM', np.float64(-13374.528372052691)),
 ('flat_type_5 ROOM', np.float64(14424.284281318432)),
 ('flat_type_EXECUTIVE', np.float64(58891.437694658045)),
 ('flat_type_MULTI-GENERATION', np.float64(166589.8969819218)),
 ('planning_area_Ang Mo Kio', np.float64(151097.53424728636)),
 ('planning_area_Bedok', np.float64(21818.657311240786)),
 ('planning_area_Bishan', np.float64(-59536.408835033464)),
 ('planning_area_Bukit Batok', np.float64(128149.45056995879)),
 ('planning_area_Bukit Merah', np.float64(-29726.60419595109)),
 ('planning_area_Bukit Panjang', np.float64(38866.94297282044)),
 ('planning_area_Bukit Timah', np.float64(20262.971015393385)),
 ('planning_area_Changi', np.float64(271091.578463686)),
 ('planning_area_Choa Chu Kang', np.float64(130379.24161084063

In [120]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import numpy as np

model_2_rmse = np.sqrt(metrics.mean_squared_error(y_test_2, y_pred_2)) # compare the RMSE with y_pred and y_test data
print(model_2_rmse)

model_2_mae = metrics.mean_absolute_error(y_test_2, y_pred_2)
print(model_2_mae)

model_2_mse = metrics.mean_squared_error(y_test_2, y_pred_2)
print(model_2_mse)

74442.4227347044
56472.178785982454
5541674302.612436


##   **Compute the Null RMSE**

In [121]:
from sklearn.metrics import mean_squared_error
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics 

# Split X and y into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=123)

# Create a NumPy array with the same shape as y_test.
y_null = np.zeros_like(y_test, dtype=float)

# Fill the array with the mean value of y_train.
y_null.fill(y_train.mean())
y_null

# Compute null RMSE, MAE & MSE
null_rmse = np.sqrt(metrics.mean_squared_error(y_test, y_null))
null_mae = metrics.mean_absolute_error(y_test, y_null)
null_mse = metrics.mean_squared_error(y_test, y_null)

print(null_rmse)
print(null_mae)
print(null_mse)

143573.95352905753
110371.42527502937
20613480131.963974


##   **Evaluation of Models**

In [122]:
# Define your metrics (replace these with your actual values)
data = {
    "Model": ["Null Model", "Model 1", "Model 2"],
    "RMSE": [null_rmse, model_1_rmse, model_2_rmse],
    "MAE": [null_mae, model_1_mae, model_2_mae]
}

# Create DataFrame
results_df = pd.DataFrame(data)

# Compute improvement (%) vs Null Model
results_df["RMSE Improvement (%)"] = (
    (results_df.loc[0, "RMSE"] - results_df["RMSE"]) / results_df.loc[0, "RMSE"] * 100
).round(2)

results_df["MAE Improvement (%)"] = (
    (results_df.loc[0, "MAE"] - results_df["MAE"]) / results_df.loc[0, "MAE"] * 100
).round(2)

# Clean up formatting
results_df = results_df.round(2)

# Display table
print("📊 Model Performance Comparison:")
display(results_df)

📊 Model Performance Comparison:


,Model,RMSE,MAE,RMSE Improvement (%),MAE Improvement (%)
0,Null Model,143573.95,110371.43,0.00,0.00
1,Model 1,61772.47,47956.77,56.98,56.55
2,Model 2,74442.42,56472.18,48.15,48.83


##   **Final Model Selection**

From the above model performance comparison, both models performed better than the Null Model. Both models' predictions are about twice as accurate as just guessing the average flat price. It is a strong performance especially for something as noisy and complex as HDB resale prices, which have many factors that might not be as statistically significant as the feature matrix (X) that we chose.

Model 1's RMSE ($61,772.47) is higher than Model 2's RMSE ($74,442.42).

This also tells us that substituting the planning_area feature with regions_mapping feature in Model 2 does not give us a higher RMSE and is not as accurate as Model 1.

Given the relative performance, we choose Model 1 as the final model in predicting target variable y (resale_price). 

In [123]:
# Include all dummy variables in feature_cols

from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

feature_cols = ["hdb_age", "floor_area_sqft", "flat_type_1 ROOM",
                "flat_type_2 ROOM", "flat_type_3 ROOM", "flat_type_5 ROOM",
                "flat_type_EXECUTIVE", "flat_type_MULTI-GENERATION",
                "planning_area_Ang Mo Kio", "planning_area_Bedok",
                "planning_area_Bishan", "planning_area_Bukit Batok",
                "planning_area_Bukit Merah", "planning_area_Bukit Panjang",
                "planning_area_Bukit Timah", "planning_area_Changi",
                "planning_area_Choa Chu Kang", "planning_area_Clementi",
                "planning_area_Downtown Core", "planning_area_Geylang",
                "planning_area_Hougang", "planning_area_Jurong East",
                "planning_area_Kallang", "planning_area_Marine Parade",
                "planning_area_Novena", "planning_area_Outram",
                "planning_area_Pasir Ris", "planning_area_Punggol",
                "planning_area_Queenstown", "planning_area_Rochor",
                "planning_area_Sembawang", "planning_area_Sengkang",
                "planning_area_Serangoon", "planning_area_Tampines",
                "planning_area_Tanglin", "planning_area_Toa Payoh",
                "planning_area_Western Water Catchment", "planning_area_Woodlands",
                "planning_area_Yishun", "new_storey_range_High (13-18)",
                "new_storey_range_Mid (7-12)", "new_storey_range_Ultra High (31+)",
                "new_storey_range_Very High (19-30)"]

X = HDB_dummies[feature_cols]
y = HDB_dummies.resale_price

X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=123) #random_state is super impt, need to fix the test & train data

model_1 = LinearRegression()
model_1.fit(X_train, y_train) # create a best fit line using train data (x, y)

y_pred = model_1.predict(X_test) # predict the Y values with x test data.

In [124]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
import joblib

##   **Preparing for Streamlit**

In [125]:
# Save trained model
joblib.dump(model_1, 'hdb_model.joblib')

print("✅ Model saved as hdb_model.joblib")

✅ Model saved as hdb_model.joblib


In [126]:
# save as pipeline

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression

# Define feature types
categorical_features = ["flat_type", "planning_area", "new_storey_range"]
numerical_features = ["hdb_age", "floor_area_sqft"]

# Combine preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
        ("num", "passthrough", numerical_features)
    ]
)

# Build pipeline
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("model", LinearRegression())
])

# Train using the *raw* dataset (not HDB_dummies)
X = HDB[["flat_type", "planning_area", "new_storey_range", "hdb_age", "floor_area_sqft"]]
y = HDB["resale_price"]

pipeline.fit(X, y)

# Save full pipeline
import joblib
joblib.dump(pipeline, "hdb_pipeline.joblib")


['hdb_pipeline.joblib']

In [127]:
# Load model
loaded_model = joblib.load('hdb_model.joblib')

# Predict using same kind of input dataframe
y_pred = loaded_model.predict(X_test)

print("✅ Loaded model works fine!")

✅ Loaded model works fine!


In [128]:
# save HDB_dummies for streamlit to load

import pandas as pd
HDB_dummies.to_csv("HDB_dummies.csv")

##   **Generating Resale Prices for test.csv**

In [129]:
import pandas as pd
import joblib

pipeline = joblib.load("hdb_pipeline.joblib")
test_data = pd.read_csv("test.csv")

C:\Users\Serene\AppData\Local\Temp\ipykernel_16260\3167239808.py:5: DtypeWarning: Columns (41) have mixed types. Specify dtype option on import or set low_memory=False.
  test_data = pd.read_csv("test.csv")


In [130]:
# rebin the storey_range for test.csv
# split the storey range values into lowest and highest storey per string

test_data[["low_storey", "high_storey"]] = test_data["storey_range"].str.split("TO", expand=True)
test_data["low_storey"] = test_data["low_storey"].astype(int)
test_data["high_storey"] = test_data["high_storey"].astype(int)

# Compute the average storey (midpoint)
test_data["avg_storey"] = (test_data["low_storey"] + test_data["high_storey"]) / 2

# Define bins and labels for rebinning
bins = [0, 6, 12, 18, 30, 100]  # upper bound of last bin = 100 (covers all)
labels = ["Low (1-6)", "Mid (7-12)", "High (13-18)",
          "Very High (19-30)", "Ultra High (31+)"]

# Create a rebinned category
test_data["new_storey_range"] = pd.cut(test_data["avg_storey"], bins=bins, labels=labels, right=True)
test_data.head()


,id,Tranc_YearMonth,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,...,sec_sch_nearest_dist,sec_sch_name,cutoff_point,affiliation,sec_sch_latitude,sec_sch_longitude,low_storey,high_storey,avg_storey,new_storey_range
0,114982,2012-11,YISHUN,4 ROOM,173,YISHUN AVE 7,07 TO 09,84.0,Simplified,1987,...,156.322353,Ahmad Ibrahim Secondary School,218,0,1.436235,103.829987,7,9,8.0,Mid (7-12)
1,95653,2019-08,JURONG WEST,5 ROOM,986C,JURONG WEST ST 93,04 TO 06,112.0,Premium Apartment,2008,...,739.371688,Jurong West Secondary School,199,0,1.335256,103.702098,4,6,5.0,Low (1-6)
2,40303,2013-10,ANG MO KIO,3 ROOM,534,ANG MO KIO AVE 10,07 TO 09,68.0,New Generation,1980,...,305.071191,Anderson Secondary School,245,0,1.374242,103.851430,7,9,8.0,Mid (7-12)
3,109506,2017-10,WOODLANDS,4 ROOM,29,MARSILING DR,01 TO 03,97.0,New Generation,1979,...,433.454591,Woodlands Secondary School,188,0,1.439183,103.774499,1,3,2.0,Low (1-6)
4,100149,2016-08,BUKIT BATOK,4 ROOM,170,BT BATOK WEST AVE 8,16 TO 18,103.0,Model A,1985,...,217.295361,Bukit Batok Secondary School,223,0,1.348351,103.740873,16,18,17.0,High (13-18)


In [131]:
test_data["Predicted"] = pipeline.predict(test_data)

In [132]:
test_data.to_csv("sample_sub_reg.csv")